In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from src import paths
import yaml

In [3]:

with open(paths.config_dir("params.yaml"), "r") as file:
    params = yaml.safe_load(file)

directory_file_prefix_map = params["directory_file_prefix_map"]
selected_directories = params["selected_directories"]
column_conventions = params["column_conventions"]
vegetation_type_conventions = params["vegetation_type_conventions"]
change_type_conventions = params["change_type_conventions"]
excluded_cols = params["excluded_cols"]

# Read NDVI time series

In [4]:
aux_filepath = paths.data_raw_dir("estable-arboreo-ts", "ESTABLE_AuxiliarFix.csv")
ndvi_filepath = paths.data_raw_dir("estable-arboreo-ts", "ESTABLE_TimeSerie_ndvi.csv")


In [5]:
aux_df = pd.read_csv(aux_filepath, index_col=0)
ndvi_df = pd.read_csv(ndvi_filepath, index_col=0)

In [12]:
df = pd.merge(aux_df, ndvi_df, left_on='IDpix',
                  right_on='IDpix', how='outer')
df = df.drop(columns=excluded_cols)
df = df.set_index(["ID", "IDpix"])
df = df.rename(columns=column_conventions)
df["vegetation_type"] = df["vegetation_type"].map(
    vegetation_type_conventions)
df["change_type"] = df["change_type"].map(change_type_conventions)
df = df[df["vegetation_type"] != "exotic"]
df = df[df["vegetation_type"] != "mixed"]

In [13]:
df

lat        lon change_type change_start vegetation_type  \
ID IDpix                                                                  
0  1     -33.843804 -71.136195      stable   03/07/2004          native   
   2     -33.843804 -71.135656      stable   03/07/2004          native   
   3     -33.843804 -71.135117      stable   03/07/2004          native   
   4     -33.843804 -71.134578      stable   03/07/2004          native   
   5     -33.843804 -71.134039      stable   03/07/2004          native   
...             ...        ...         ...          ...             ...   
59 1528  -33.398868 -71.358258      stable   22/01/2005          native   
   1529  -33.399138 -71.360145      stable   22/01/2005          native   
   1530  -33.398868 -71.357989      stable   22/01/2005          native   
   1531  -33.399138 -71.359875      stable   22/01/2005          native   
   1532  -33.398599 -71.361223      stable   22/01/2005          native   

          2000-01-03  2000-01-11  2000-01-18  2000-01-26  2000-01-27  ...  \
ID IDpix                                                              ...   
0  1             NaN    0.707001         NaN         NaN    0.723551  ...   
   2             NaN    0.730423         NaN         NaN    0.720345  ...   
   3             NaN    0.746536         NaN         NaN    0.741756  ...   
   4             NaN    0.750166         NaN         NaN    0.741654  ...   
   5             NaN    0.765939         NaN         NaN    0.749571  ...   
...              ...         ...         ...         ...         ...  ...   
59 1528          NaN    0.850285    0.794070         NaN         NaN  ...   
   1529          NaN    0.850285    0.747522         NaN         NaN  ...   
   1530          NaN    0.790671    0.765448         NaN         NaN  ...   
   1531          NaN    0.796872    0.823520         NaN         NaN  ...   
   1532          NaN    0.838390    0.815240         NaN         NaN  ...   

          2022-12-01  2022-12-02  2022-12-08  2022-12-09  2022-12-14  \
ID IDpix                                                               
0  1        0.700959         NaN         NaN    0.723368    0.615958   
   2        0.728630         NaN         NaN    0.732564    0.655706   
   3        0.748324         NaN         NaN    0.764361    0.695667   
   4        0.752141         NaN         NaN    0.765921    0.735234   
   5        0.756909         NaN         NaN    0.761045    0.710822   
...              ...         ...         ...         ...         ...   
59 1528     0.791864         NaN    0.796135         NaN         NaN   
   1529     0.791864         NaN    0.760986         NaN         NaN   
   1530     0.746991         NaN    0.778123         NaN         NaN   
   1531     0.789701         NaN    0.814714         NaN         NaN   
   1532     0.822292         NaN    0.830918         NaN         NaN   

          2022-12-16  2022-12-17  2022-12-19  2022-12-24  2022-12-25  
ID IDpix                                                              
0  1             NaN    0.683760         NaN         NaN    0.660891  
   2             NaN    0.755396         NaN         NaN    0.734281  
   3             NaN    0.774370         NaN         NaN    0.749191  
   4             NaN    0.772314         NaN         NaN    0.746010  
   5             NaN    0.760751         NaN         NaN    0.744337  
...              ...         ...         ...         ...         ...  
59 1528          NaN         NaN    0.624895    0.794414         NaN  
   1529          NaN         NaN    0.551228    0.761138         NaN  
   1530          NaN         NaN    0.593962    0.780284         NaN  
   1531          NaN         NaN    0.772596    0.817876         NaN  
   1532          NaN         NaN    0.670126    0.833752         NaN  

[1532 rows x 1900 columns]

In [18]:
dfs = []

for directory in selected_directories:
    filename_prefix = directory_file_prefix_map[directory]
    change_type = filename_prefix.lower()

    ndvi_filename = filename_prefix + "_TimeSerie_ndvi.csv"
    aux_filename = filename_prefix + "_AuxiliarFix.csv"

    ndvi_filepath = paths.data_raw_dir(directory, ndvi_filename)

    aux_filepath = paths.data_raw_dir(directory, aux_filename)

    aux_df = pd.read_csv(aux_filepath, index_col=0)
    ndvi_df = pd.read_csv(ndvi_filepath, index_col=0)

    df = pd.merge(aux_df, ndvi_df, left_on='IDpix',
                  right_on='IDpix', how='outer')
    df = df.drop(columns=excluded_cols)
    df = df.set_index(["ID", "IDpix"])
    df = df.rename(columns=column_conventions)
    df["vegetation_type"] = df["vegetation_type"].map(
        vegetation_type_conventions)
    df["change_type"] = df["change_type"].map(change_type_conventions)
    df = df[df["vegetation_type"] != "exotic"]

    out_directory = change_type_conventions[change_type]
    out_filename = change_type_conventions[change_type] + "_ndvi.csv"
    out_path = paths.data_interim_dir(out_directory, out_filename)

    if not out_path.exists():
        out_path.parent.mkdir(parents=True, exist_ok=True)
        out_path.touch()

    df.to_csv(out_path, index=False)

KeyboardInterrupt: 